# Imports

In [1]:
import numpy as np
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, normalize

from src.utils import arr_to_dict, fourier_transform, metric_report

# Data preparation

Применим к данным следующие методы процессинга:

- Преобразование Фурье и последующее использование амплитудного спектра.
- Нормализация полученных спектров.

In [2]:
data_arr = np.load("data/filtered_data.npy", allow_pickle=True)
data = arr_to_dict(data_arr)

test_data_arr = np.load("data/two_stage_test.npy", allow_pickle=True)
test_data = arr_to_dict(test_data_arr)

In [3]:
# Объединение 2-х преобразованных сигналов в один массив
X = np.hstack(
    (
        fourier_transform(data['signal_1']),
        fourier_transform(data['signal_2']),
    )
)
X_test = np.hstack(
    (
        fourier_transform(test_data['signal_1']),
        fourier_transform(test_data['signal_2']),
    )
)

# Бинаризация таргетов
le = LabelEncoder()
y = le.fit_transform(data['label'])
y_test = le.transform(test_data['label'])

# Сплит данных на train/val
split = np.load("data/splitted_idx.npy", allow_pickle=True)
X_train, X_val = X[split['train']],  X[split['val']]
y_train, y_val = y[split['train']],  y[split['val']]

# Нормализация данных
X_train = normalize(X_train)
X_val = normalize(X_val)
X_test = normalize(X_test)

del data_arr
del data

# Model

Были использованы более простые модели классического МЛ. Наилучший результат показал бустинг со следующими параметрами:
- `max_depth = 4`. Был взят меньше для борьбы с переобучением
- `scale_pos_weight = 1`. Борьба с дисбалансом классов.
- `sampling_method = gradient_based`.
- `objective = binary:logitraw`.

***XGboost***

In [13]:
xgb_clf = xgb.XGBClassifier(
    tree_method='gpu_hist',
    max_depth=4,
    scale_pos_weight=0.01,
    sampling_method='gradient_based',
    objective='binary:logitraw',
    random_state=42,
)
xgb_clf.fit(X_train, y_train)

[16:41:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:1205: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='binary:logitraw',
              predictor='auto', random_state=42, reg_alpha=0, ...)

In [14]:
metric_report(
    xgb_clf,
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
    le.classes_
)

Train
               precision    recall  f1-score   support

Sleep stage 4       0.93      1.00      0.96      2388
Sleep stage W       1.00      1.00      1.00    162109

     accuracy                           1.00    164497
    macro avg       0.96      1.00      0.98    164497
 weighted avg       1.00      1.00      1.00    164497
 

Val
               precision    recall  f1-score   support

Sleep stage 4       0.88      0.98      0.93      1056
Sleep stage W       1.00      1.00      1.00     69443

     accuracy                           1.00     70499
    macro avg       0.94      0.99      0.96     70499
 weighted avg       1.00      1.00      1.00     70499
 

Test
               precision    recall  f1-score   support

Sleep stage 4       1.00      0.96      0.98      8208
Sleep stage W       0.96      1.00      0.98      7488

     accuracy                           0.98     15696
    macro avg       0.98      0.98      0.98     15696
 weighted avg       0.98      0.98    

***LogisticRegression***

In [15]:
logreg_clf = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
logreg_clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

In [16]:
metric_report(
    logreg_clf,
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
    le.classes_
)

Train
               precision    recall  f1-score   support

Sleep stage 4       0.53      0.99      0.69      2388
Sleep stage W       1.00      0.99      0.99    162109

     accuracy                           0.99    164497
    macro avg       0.77      0.99      0.84    164497
 weighted avg       0.99      0.99      0.99    164497
 

Val
               precision    recall  f1-score   support

Sleep stage 4       0.53      0.99      0.69      1056
Sleep stage W       1.00      0.99      0.99     69443

     accuracy                           0.99     70499
    macro avg       0.76      0.99      0.84     70499
 weighted avg       0.99      0.99      0.99     70499
 

Test
               precision    recall  f1-score   support

Sleep stage 4       1.00      0.95      0.97      8208
Sleep stage W       0.95      1.00      0.97      7488

     accuracy                           0.97     15696
    macro avg       0.97      0.97      0.97     15696
 weighted avg       0.97      0.97    

# Conclusions

Давайте рассмотрим, какие эксперименты были проведены и каков их результат:

1. Использованные модели. Было решено провести эксперименты с двумя моделями:

    - LogisticRegression, как самый простой подход к решению задачи.
    - XGboost. Был выбран в качестве более сильной модели, так как, как правило, дает неплохие результаты в сравнении с остальными подходами классического МЛ.

2. Предобработка данных: 
    - Применение моделей к сырому, никак не обработанному сигналу.
    - Скейлинг, нормализация сырого сигнала.
    - Преобразование Фурье.
    - Преобразование Фурье + нормализация.

Обе модели обучались с учетом дисбаланса классов, использовались соответсвующие аргументы моделей: `scale_pos_weight` для `XGBClassifier` и `class_weight` для `LogisticRegression`.

Без использования какой-либо обработки сигнала обе модели показали очень плохой результат на тестовых данных. Бустинг смог достичь высоких результатов на train и val выборках, логистичсекая регрессия показала себя хорошо только на трейне.

Скейлинг и нормализация незначительно улучишили результаты на тесте только для бустинга, но качество было далеко от бейзлайна. Использование только преобразования Фурье ни привело к улучшению результатов. Все это можно обяснить тем, что train и test выборки из разных распределений, что было выявлено на этапе `EDA`.

Резко к улучшению качества модели на тесте приводит нормализация результатов преобразования Фурье. Получаем следующие результаты на тестовой выборке:

1. XGboost:
    - Sleep stage 4: Precision - 1.00, Recall - 0.96
    - Sleep stage W: Precision - 0.96, Recall - 1.00
    
2. LogisticRegression:
    - Sleep stage 4: Precision - 1.00, Recall - 0.95
    - Sleep stage W: Precision - 0.95, Recall - 1.00

Можно заметить, что качество моделей практически одинаково, однако логистическая регрессия, в отличие от бустинга, имеет плохие метрики на обучающей выборке. 

Таким образом, для данного сетапа задачу можно хорошо решить, используя методы классического МЛ, но правильно предобработав данные.

В рамках backnone задачи в дальнейшем можно рассмотреть нейросетевой подход. Использовать другие методы обработки сигнала, например, MFCC. В качестве модели можно использовать [CNN](https://arxiv.org/pdf/1904.10255.pdf), обученную на большом корпусе EEG данных, которая потом будет использоваться в качестве feature extractor.